# Extrahera information från domar

### OpenAI API
Olika sätt att använda GPT för att extrahera information:
1. Skicka upp PDF:er till OpenAI
2. Komvertera PDF till text och skicka med texten i prompten
3. Konvertera PDF till bild och skicka till GPT-vision?

### Vilken information vill vi ha ut?
Dom, datum, bifaller/avslår, om avslår - anledning

In [3]:
from openai import OpenAI
import openai
import pandas as pd
import os
import io
import pdfplumber
from dotenv import load_dotenv
from pdf2image import convert_from_path
import base64
import json

load_dotenv()

POPPLER_PATH = os.getenv("POPPLER_PATH")
OPENAI_ORG = os.getenv("OPENAI_ORG")

openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [5]:
data_folder = os.path.join("data")

# Loop over files in data folder
for filename in os.listdir(data_folder):
    file_path = os.path.join(data_folder, filename)
    print(file_path)

data\Uppsala FR 6124-18 Dom 2018-12-05.pdf
data\Uppsala FR 982-15 Dom 2015-04-14.pdf


In [7]:
# Convert PDF to images
images = convert_from_path(file_path, poppler_path=POPPLER_PATH)

for i, image in enumerate(images):
    fname = "image" + str(i) + ".png"
    image.save(fname, "PNG")

# OpenAI filer

### Ladda upp filer till OpenAI

In [9]:
# Ladda upp fil till OpenAI
client.files.create(
    file=open(file_path, 'rb'),
    purpose='assistants'
)

FileObject(id='file-UCXUrII44ag2VXKjarVzKdBl', bytes=236179, created_at=1701759361, filename='Uppsala FR 982-15 Dom 2015-04-14.pdf', object='file', purpose='assistants', status='processed', status_details=None)

## Lista filer

In [15]:
for file in client.files.list():
    print(f"File id: {file.id}\nFilename: {file.filename}\n")

File id: file-UCXUrII44ag2VXKjarVzKdBl
Filename: Uppsala FR 982-15 Dom 2015-04-14.pdf



In [2]:
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role":"system", "content":"Du är en hjälpsam AI-assistent som läser igenom domar och extraherar information från dem."},
        {"role":"user", "content":f"Här kommer domen, extrahera information om domen och returnera ett json-objekt. Jag vill ha datum för domen, namn för anklagade, utfall av domen."},
    ]
)

In [17]:
for model in client.models.list():
    print(model.id)

text-search-babbage-doc-001
curie-search-query
text-davinci-003
text-search-babbage-query-001
babbage
babbage-search-query
text-babbage-001
text-similarity-davinci-001
davinci-similarity
code-davinci-edit-001
curie-similarity
babbage-search-document
curie-instruct-beta
text-search-ada-doc-001
davinci-instruct-beta
gpt-3.5-turbo-instruct
text-similarity-babbage-001
text-search-davinci-doc-001
babbage-similarity
text-embedding-ada-002
davinci-search-query
text-similarity-curie-001
text-davinci-001
gpt-3.5-turbo-instruct-0914
text-search-davinci-query-001
ada-search-document
ada-code-search-code
babbage-002
davinci-002
gpt-3.5-turbo-0613
davinci-search-document
curie-search-document
babbage-code-search-code
text-search-ada-query-001
code-search-ada-text-001
babbage-code-search-text
gpt-3.5-turbo-0301
code-search-babbage-code-001
ada-search-query
gpt-3.5-turbo
ada-code-search-text
gpt-4-0613
tts-1-hd
gpt-3.5-turbo-16k-0613
text-search-curie-query-001
text-davinci-002
text-davinci-edit-001


In [37]:
os.listdir("dom1")

['image0.png',
 'image1.png',
 'image10.png',
 'image2.png',
 'image3.png',
 'image4.png',
 'image5.png',
 'image6.png',
 'image7.png',
 'image8.png',
 'image9.png']

## Använd PDFplumber för att göra om pdf:er till texter

In [100]:

# Open and read the PDF
for filename in os.listdir("Domar - pdf"):
    # Initialize a text string
    text = ""

    pdf = pdfplumber.open(os.path.join("Domar - pdf/", filename))
    for page in pdf.pages:
        if page.page_number > 1:
            page = page.crop((0, page.height*.1, page.width, page.height*.9))
        else:
            page = page.crop((0, 0, page.width, page.height*.9))
        # crop 2 cm from bottom
        text += '\n\n' + page.extract_text()

    pdf.close()

    # save to file
    with open(os.path.join("Domar - txt/", filename.replace(".pdf", ".txt")), "w", encoding="utf-8") as f:
        f.write(text)

In [4]:
with open("Domar - txt/Uppsala FR 982-15 Dom 2015-04-14.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [5]:
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_format={"type":"json_object"},
    messages=[
        {"role": "system", 
         "content": "Du är en hjälpsam AI-assistent som läser igenom domar och extraherar information från dem."},
        {"role": "user", 
         "content": f"""
         Jag har tre frågor om domen och vill att du svarar på dem i JSON-format med följande nycklar:
         'Datum': Vilket datum är domen? Svara ENDAST med datum i formatet YYYY-MM-DD.
         'Beslut': Avslår eller bifaller förvaltningsrätten ansökan om LVU? Svara ENDAST med "Avslår" eller "Bifaller".
         'Anledning': Om domen avslår ansökan, varför?.

         Här kommer domen, svara ENDAST med json-objektet.:

         {text}
         """}
    ]
)


print(f"""
Antal tokens:
Prompt tokens: {response.usage.prompt_tokens}
Completion tokens: {response.usage.completion_tokens}
Total tokens: {response.usage.total_tokens}
""")

print(f"""
Uppskattad kostnad:
$ {round(response.usage.prompt_tokens/1000 * 0.01 + response.usage.completion_tokens/1000 * 0.03, 3)}
""")

print(response.choices[0].message.content)


Antal tokens:
Prompt tokens: 6438
Completion tokens: 118
Total tokens: 6556


Uppskattad kostnad:
$ 0.068

{
  "Datum": "2015-04-14",
  "Beslut": "Avslår",
  "Anledning": "Stief Al Wanis situation har förändrats på ett sådant sätt att det, trots den korta tid som förflutit, inte längre finns konkreta omständigheter som visar att det föreligger ett sådant socialt nedbrytande beteende som utgör en påtaglig risk för Stief Al Wanis hälsa eller utveckling."
}


In [158]:
response.choices[0].message.content.replace("\n  ", "")

'{"Datum": "2015-04-14","Beslut": "Avslår","Varför": "Situationen har förändrats på ett sådant sätt att det inte längre finns konkreta omständigheter som visar att det föreligger ett sådant socialt nedbrytande beteende som utgör en påtaglig risk för Stief Al Wanis hälsa eller utveckling. Skolan och föräldrarna har arbetat samman för att ge honom den nödvändiga uppsikten och stödet han behöver, och därmed bedöms risken för hans hälsa eller utveckling vara avlägsen."\n}'

In [8]:
json.loads(response.choices[0].message.content)

{'Datum': '2015-04-14',
 'Beslut': 'Avslår',
 'Anledning': 'Stief Al Wanis situation har förändrats på ett sådant sätt att det, trots den korta tid som förflutit, inte längre finns konkreta omständigheter som visar att det föreligger ett sådant socialt nedbrytande beteende som utgör en påtaglig risk för Stief Al Wanis hälsa eller utveckling.'}

In [6]:
# prettyrpint
print(json.dumps(json.loads(response.choices[0].message.content), indent=4, sort_keys=True, ensure_ascii=False))

{
    "Anledning": "Stief Al Wanis situation har f\u00f6r\u00e4ndrats p\u00e5 ett s\u00e5dant s\u00e4tt att det, trots den korta tid som f\u00f6rflutit, inte l\u00e4ngre finns konkreta omst\u00e4ndigheter som visar att det f\u00f6religger ett s\u00e5dant socialt nedbrytande beteende som utg\u00f6r en p\u00e5taglig risk f\u00f6r Stief Al Wanis h\u00e4lsa eller utveckling.",
    "Beslut": "Avsl\u00e5r",
    "Datum": "2015-04-14"
}


In [1]:
pd.DataFrame.from_dict(json.loads(response.choices[0].message.content), orient="index")

NameError: name 'pd' is not defined

In [117]:

print(f"""
Completion tokens: {response.usage.completion_tokens}
Prompt tokens: {response.usage.prompt_tokens}
Total tokens: {response.usage.total_tokens}
""")


Completion tokens: 116
Prompt tokens: 7087
Total tokens: 7203



## GPT-vision multiple image inputs

In [45]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


# apply above function to encode all images in folder "dom1" and save them in folder "dom1_encoded"
for filename in os.listdir("dom1"):
    file_path = os.path.join("dom1", filename)
    #print(file_path)
    base64_image = encode_image(file_path)
    #print(base64_image)
    with open(os.path.join("dom1_encoded", filename), "w") as f:
        f.write(base64_image)

In [70]:
# read 64base encoded image
with io.open("dom1_encoded/image0.png", "r") as image_file:
    encoded_string = image_file.read()

encoded_string

'...'

In [72]:
# read 64base encoded images from folder "dom1_encoded" and add them to list
images = []
for filename in os.listdir("dom1_encoded"):
    file_path = os.path.join("dom1_encoded", filename)
    with io.open(file_path, "r") as image_file:
        encoded_string = image_file.read()
        images.append(encoded_string)

In [ ]:
# do the above for all encoded strings in the list "images"
image_list = []
for encoded_string in images:
    image_dict = {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{encoded_string}",
        },
    }
    image_list.append(image_dict)


In [ ]:
#convert list to json
import json
json_images = json.dumps(image_list)
json_images

In [81]:



response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", 
         "text": "Extrahera domslutet"},
        json_images
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-vision-preview in organization org-U7PUAm1K5MYqIGGJRsEcGbAq on tokens per min (TPM): Limit 150000, Requested 1353096. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

## OpenAI Assistants

In [19]:

my_assistant = client.beta.assistants.create(
    description="Du är en hjälpsam AI-assistent som läser igenom domar och extraherar information från dem.",
    name="Domarassistenten",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=["file-UCXUrII44ag2VXKjarVzKdBl"]
)

print(my_assistant)

Assistant(id='asst_OKNSL62TV50WDGo9t067TO6o', created_at=1701763403, description=None, file_ids=['file-UCXUrII44ag2VXKjarVzKdBl'], instructions='Du är en hjälpsam AI-assistent som läser igenom domar och extraherar information från dem.', metadata={}, model='gpt-4-1106-preview', name='Domarassistenten', object='assistant', tools=[ToolRetrieval(type='retrieval')])


In [24]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Beskriv vad filen handlar om",
      "file_ids": ["file-UCXUrII44ag2VXKjarVzKdBl"]
    }
  ]
)

print(thread)

Thread(id='thread_hEcIEXETh6Z03c0uqczsKyCi', created_at=1701763793, metadata={}, object='thread')


In [28]:
run = client.beta.threads.runs.create(
  thread_id="thread_hEcIEXETh6Z03c0uqczsKyCi",
  assistant_id="asst_OKNSL62TV50WDGo9t067TO6o",
  model="gpt-4-1106-preview",
  instructions="Extrahera information om domen och returnera ett json-objekt. Jag vill ha datum för domen, namn för anklagade, utfall av domen.",
  tools=[{"type": "retrieval"}]
)

print(run)

Run(id='run_efeSC2HjjRHXYKlfrzksup6Y', assistant_id='asst_OKNSL62TV50WDGo9t067TO6o', cancelled_at=None, completed_at=None, created_at=1701763909, expires_at=1701764509, failed_at=None, file_ids=['file-UCXUrII44ag2VXKjarVzKdBl'], instructions='Extrahera information om domen och returnera ett json-objekt. Jag vill ha datum för domen, namn för anklagade, utfall av domen.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_hEcIEXETh6Z03c0uqczsKyCi', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [31]:
run = client.beta.threads.runs.retrieve(
  thread_id="thread_hEcIEXETh6Z03c0uqczsKyCi",
  run_id="run_efeSC2HjjRHXYKlfrzksup6Y"
)
print(run)

Run(id='run_efeSC2HjjRHXYKlfrzksup6Y', assistant_id='asst_OKNSL62TV50WDGo9t067TO6o', cancelled_at=None, completed_at=1701763936, created_at=1701763909, expires_at=None, failed_at=None, file_ids=['file-UCXUrII44ag2VXKjarVzKdBl'], instructions='Extrahera information om domen och returnera ett json-objekt. Jag vill ha datum för domen, namn för anklagade, utfall av domen.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1701763917, status='completed', thread_id='thread_hEcIEXETh6Z03c0uqczsKyCi', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


In [32]:
run = client.beta.threads.create_and_run(
  assistant_id="asst_OKNSL62TV50WDGo9t067TO6o",
  thread={
    "messages": [
      {"role": "user", "content": "Explain deep learning to a 5 year old."}
    ]
  }
)

In [34]:
run = client.beta.threads.runs.cancel(
  thread_id="thread_CAzju5uWLgpBhGx3G2jU8D7O",
  run_id="run_ZzE01oK4SnKZcW1NX1I7DLGB"
)
print(run)

BadRequestError: Error code: 400 - {'error': {'message': "Cannot cancel run with status 'completed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}